In [1]:
!git clone https://github.com/MorningStarTM/Mixture-of-LLM-Expert.git

Cloning into 'Mixture-of-LLM-Expert'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 97 (delta 43), reused 89 (delta 39), pack-reused 0
Unpacking objects: 100% (97/97), 16.15 KiB | 636.00 KiB/s, done.


In [2]:
%cd /kaggle/working/Mixture-of-LLM-Expert

/kaggle/working/Mixture-of-LLM-Expert


In [3]:
!ls

LICENSE  MoE  Models  README.md  Tokenizer  Utils


In [10]:
import os
import re
import json
from MoE import SparseMoELanguageModel, kaiming_init_weights
from Utils import model_params
from Tokenizer import WordLevelTokenizer, extract_and_save_text, LetterLevelTokenizer
import torch

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device
max_iters = 100000
learning_rate = 3e-4
block_size = 8
batch_size = 8
eval_iters = 500
n_emb = 384
n_layers = 6
n_head = 6

In [6]:
csv_file_path = '/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv'  # Replace with your CSV file path
output_directory = '/kaggle/working/'  # Replace with your desired output directory

extract_and_save_text(csv_file_path, output_directory)

In [7]:
with open("/kaggle/working/articles.txt", "rb") as txt:
    texts = txt.read()

In [8]:
chars = sorted(list(set(texts.decode())))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz|}~ £¥©«¬­®°±´µ·¹º»¼½¾ÁÂÄÅÈÉÓÖ×ØÛÜßàáâãäåçèéêëíîïðñòóôö÷øúûüāăćčēęğīİıłńōřŚśşŠšūŷžș˚ΕΗΘΛΝΣΤΦβμ܎ღạ​‎‏‐‑–—‘’“”•…‪‬ ⁄⁰€™↑■●♥❤ﬂ️－�💗
218


In [11]:
tokenizer = LetterLevelTokenizer()

In [12]:
temp = "Im the President of the America."

tokens = tokenizer.tokenize(temp)
print("Tokens:", tokens)  # Output: Tokens: [index values representing each word]

original_text = tokenizer.detokenize(tokens)
print("Detokenized text:", original_text)

Tokens: [34, 12, 94, 19, 7, 4, 94, 41, 17, 4, 18, 8, 3, 4, 13, 19, 94, 14, 5, 94, 19, 7, 4, 94, 26, 12, 4, 17, 8, 2, 0, 75]
Detokenized text: Im the President of the America.


In [13]:
data = tokenizer.tokenize(texts.decode())

In [14]:
encoded_data = torch.tensor(data, dtype=torch.long)
print(encoded_data.shape, encoded_data.dtype)

torch.Size([45489315]) torch.int64


In [15]:
n = int(0.9*len(encoded_data))
train_data = encoded_data[:n]
val_data = encoded_data[n:]

print(f"Training tokens : {len(train_data)}  --- Validation tokens : {len(val_data)}")

Training tokens : 40940383  --- Validation tokens : 4548932


In [16]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([30]) the target: 21
when input is tensor([30, 21]) the target: 4
when input is tensor([30, 21,  4]) the target: 17
when input is tensor([30, 21,  4, 17]) the target: 94
when input is tensor([30, 21,  4, 17, 94]) the target: 13
when input is tensor([30, 21,  4, 17, 94, 13]) the target: 14
when input is tensor([30, 21,  4, 17, 94, 13, 14]) the target: 19
when input is tensor([30, 21,  4, 17, 94, 13, 14, 19]) the target: 8


In [17]:
torch.manual_seed(1337)

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(encoded_data) - block_size, (batch_size,))
    x = torch.stack([encoded_data[i:i+block_size] for i in ix])
    y = torch.stack([encoded_data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("Inputs: ")
print(xb)
print("Targets: ")
print(yb)

Inputs: 
tensor([[18, 94,  1,  8, 17, 19,  7,  3],
        [46, 13,  8, 21,  4, 17, 18,  8],
        [19, 94,  7,  0,  3, 94, 19,  7],
        [ 8, 14, 13, 94, 17,  4, 12,  0],
        [ 6, 14,  8, 13,  6, 94,  0, 94],
        [ 3,  4,  0, 94, 22,  7,  4, 17],
        [19,  4, 15,  7,  4, 13, 94, 44],
        [75, 68, 94, 31, 14, 17, 94,  2]])
Targets: 
tensor([[94,  1,  8, 17, 19,  7,  3,  0],
        [13,  8, 21,  4, 17, 18,  8, 19],
        [94,  7,  0,  3, 94, 19,  7,  4],
        [14, 13, 94, 17,  4, 12,  0,  8],
        [14,  8, 13,  6, 94,  0, 94, 53],
        [ 4,  0, 94, 22,  7,  4, 17,  4],
        [ 4, 15,  7,  4, 13, 94, 44, 12],
        [68, 94, 31, 14, 17, 94,  2, 14]])


In [19]:
model = SparseMoELanguageModel(vocab_size)
model.apply(kaiming_init_weights)
model.to(device)

SparseMoELanguageModel(
  (token_embedding_table): Embedding(218, 368)
  (position_embedding_table): Embedding(32, 368)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (key): Linear(in_features=368, out_features=46, bias=False)
            (query): Linear(in_features=368, out_features=46, bias=False)
            (value): Linear(in_features=368, out_features=46, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=368, out_features=368, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (smoe): SparseMoE(
        (router): NoisyTopkRouter(
          (topkroute_linear): Linear(in_features=368, out_features=8, bias=True)
          (noise_linear): Linear(in_features=368, out_features=8, bias=True)
        )
        (experts): ModuleList(
          (0-7): 8 x Expert(
            (net): Sequential(
            

# Model paramters

In [20]:
model_params(model)

Total parameters: 74,023,546
Trainable parameters: 74,023,546
Non-trainable parameters: 0


# Support Function

In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X, Y = X.to(device), Y.to(device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Training

In [22]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"steps: {iter} train loss: {losses['train']} val loss: {losses['val']}")
        
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb = yb.to(device)

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

steps: 0 train loss: 6.797354698181152 val loss: 6.806445598602295
steps: 500 train loss: 2.5807409286499023 val loss: 2.567488193511963
steps: 1000 train loss: 2.4178872108459473 val loss: 2.414222240447998
steps: 1500 train loss: 2.3510682582855225 val loss: 2.3578083515167236
steps: 2000 train loss: 2.359004020690918 val loss: 2.334740161895752
steps: 2500 train loss: 2.2673230171203613 val loss: 2.250094413757324
steps: 3000 train loss: 2.2315187454223633 val loss: 2.232961654663086
steps: 3500 train loss: 2.2234010696411133 val loss: 2.2287089824676514
steps: 4000 train loss: 2.1672251224517822 val loss: 2.1592421531677246
steps: 4500 train loss: 2.1866283416748047 val loss: 2.18245792388916
steps: 5000 train loss: 2.127903699874878 val loss: 2.1395297050476074
steps: 5500 train loss: 2.1221275329589844 val loss: 2.1430952548980713
steps: 6000 train loss: 2.1232824325561523 val loss: 2.121630907058716
steps: 6500 train loss: 2.103116273880005 val loss: 2.1051766872406006
steps: 70

KeyboardInterrupt: 

In [47]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_word = model.generate(context, max_new_tokens=50)
seq = " Batman is the "
for i in generated_word.tolist():
    seq = seq + ""+tokenizer.detokenize(i)

In [48]:
seq

' Batman is the avy Safle irzistmilittin secy h Careven,hites  Uthr'

In [49]:
context

tensor([[0]], device='cuda:0')